In [1]:
!nvidia-smi                             # GPU 정보 확인
!head /proc/cpuinfo                     # CPU 정보 확인
!/usr/local/cuda/bin/nvcc --version     # CUDA 정보 확인

Thu May 20 00:33:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# (안함) cuDNN 설치 파일 경로로 이동
# cd (경로)

In [ ]:
# (안함) cuDNN 설치
# !tar -xzvf cudnn-11.0-linux-x64-v8.0.5.39.tgz -C /usr/local/

In [ ]:
# (안함) cuDNN 설치 확인
# !chmod a+r /usr/local/cuda/include/cudnn.h
# !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

# darknet 설치
- YOLOv4 튜토리얼 코드 그대로 사용
- 파일 경로들만 조금 다름

In [ ]:
# darknet 설치하려는 경로로 이동
# 본인 구글 드라이브 內
%cd /content/drive/Shareddrives/mulcam_ai/yolov4_tiny

In [ ]:
# darknet 설치
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# openCV, CUDA, cuDNN 사용을 위한 설정
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
# cuDNN_HALF는 NVIDIA 상위 GPU 기종에서 사용가능

In [ ]:
!make

## 선택사항

In [ ]:
# 결과물 확인 등을 위한 함수 정의
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

# 튜토리얼

In [ ]:
# 튜토리얼을 위한 weights 파일 다운로드
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# 기본 테스트 이미지(person) 사용
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
# show image using our helper function
imShow('predictions.jpg')

# Custom Training

In [ ]:
# data 다운로드 받을 경로
%cd /content/drive/Shareddrives/mulcam_ai/yolov4_tiny/darknet/data

In [ ]:
# data 다운로드
!git clone https://github.com/EyesOn-FinalProject/bus_img.git

In [ ]:
# 학습 시작하기 위한 경로
%cd /content/drive/Shareddrives/mulcam_ai/yolov4_tiny/darknet

In [4]:
!chmod +x ./darknet

In [10]:
# 학습 시작
# ./darknet detector train <.data> <.cfg> <.weights> -dont_show -map
!./darknet detector train cfg/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -dont_show -map

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.743138), count: 3, class_loss = 0.216179, iou_loss = 4.380197, total_loss = 4.596376 
 total_bbox = 1322209, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.929122), count: 3, class_loss = 0.000003, iou_loss = 0.361133, total_loss = 0.361136 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.912481), count: 5, class_loss = 0.000102, iou_loss = 5.801258, total_loss = 5.801360 
 total_bbox = 1322217, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.895385), count: 6, class_loss = 0.001216, iou_loss = 0.800989, total_loss = 0.802206 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.895206), count: 3, class_loss = 0.100661, iou_loss = 1.627564, total_loss = 1.728225 
 total_bbox = 132222

# Test

In [8]:
!./darknet detector demo cfg/obj.data cfg/yolov4-tiny-obj.cfg backup/yolov4-tiny-obj_best.weights data/green_test2.mp4 -dont_show -out_filename green_test2_out_0520_v4_tiny_best.avi

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
Demo
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8

In [9]:
!./darknet detector demo cfg/obj.data cfg/yolov4-tiny-obj.cfg backup/yolov4-tiny-obj_last.weights data/green_test2.mp4 -dont_show -out_filename green_test2_out_0520_v4_tiny_last.avi

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
Demo
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8